In [0]:
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division

import warnings

from keras.models import Model
from keras.layers.core import Dense, Dropout, Activation, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers.pooling import AveragePooling2D, MaxPooling2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers import Input
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.utils.layer_utils import convert_all_kernels_in_model, convert_dense_weights_data_format
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.applications.imagenet_utils import decode_predictions
import keras.backend as K

# from subpixel import SubPixelUpscaling

DENSENET_121_WEIGHTS_PATH = r'https://github.com/titu1994/DenseNet/releases/download/v3.0/DenseNet-BC-121-32.h5'
DENSENET_161_WEIGHTS_PATH = r'https://github.com/titu1994/DenseNet/releases/download/v3.0/DenseNet-BC-161-48.h5'
DENSENET_169_WEIGHTS_PATH = r'https://github.com/titu1994/DenseNet/releases/download/v3.0/DenseNet-BC-169-32.h5'
DENSENET_121_WEIGHTS_PATH_NO_TOP = r'https://github.com/titu1994/DenseNet/releases/download/v3.0/DenseNet-BC-121-32-no-top.h5'
DENSENET_161_WEIGHTS_PATH_NO_TOP = r'https://github.com/titu1994/DenseNet/releases/download/v3.0/DenseNet-BC-161-48-no-top.h5'
DENSENET_169_WEIGHTS_PATH_NO_TOP = r'https://github.com/titu1994/DenseNet/releases/download/v3.0/DenseNet-BC-169-32-no-top.h5'

def preprocess_input(x, data_format=None):
    """Preprocesses a tensor encoding a batch of images.
    # Arguments
        x: input Numpy tensor, 4D.
        data_format: data format of the image tensor.
    # Returns
        Preprocessed tensor.
    """
    if data_format is None:
        data_format = K.image_data_format()
    assert data_format in {'channels_last', 'channels_first'}

    if data_format == 'channels_first':
        if x.ndim == 3:
            # 'RGB'->'BGR'
            x = x[::-1, ...]
            # Zero-center by mean pixel
            x[0, :, :] -= 103.939
            x[1, :, :] -= 116.779
            x[2, :, :] -= 123.68
        else:
            x = x[:, ::-1, ...]
            x[:, 0, :, :] -= 103.939
            x[:, 1, :, :] -= 116.779
            x[:, 2, :, :] -= 123.68
    else:
        # 'RGB'->'BGR'
        x = x[..., ::-1]
        # Zero-center by mean pixel
        x[..., 0] -= 103.939
        x[..., 1] -= 116.779
        x[..., 2] -= 123.68

    x *= 0.017 # scale values

    return x

def __create_dense_net(nb_classes, img_input, include_top, depth=40, nb_dense_block=3, growth_rate=12, nb_filter=-1,
                       nb_layers_per_block=-1, bottleneck=False, reduction=0.0, dropout_rate=None, weight_decay=1e-4,
                       subsample_initial_block=False, activation='softmax'):
    ''' Build the DenseNet model
    Args:
        nb_classes: number of classes
        img_input: tuple of shape (channels, rows, columns) or (rows, columns, channels)
        include_top: flag to include the final Dense layer
        depth: number or layers
        nb_dense_block: number of dense blocks to add to end (generally = 3)
        growth_rate: number of filters to add per dense block
        nb_filter: initial number of filters. Default -1 indicates initial number of filters is 2 * growth_rate
        nb_layers_per_block: number of layers in each dense block.
                Can be a -1, positive integer or a list.
                If -1, calculates nb_layer_per_block from the depth of the network.
                If positive integer, a set number of layers per dense block.
                If list, nb_layer is used as provided. Note that list size must
                be (nb_dense_block + 1)
        bottleneck: add bottleneck blocks
        reduction: reduction factor of transition blocks. Note : reduction value is inverted to compute compression
        dropout_rate: dropout rate
        weight_decay: weight decay rate
        subsample_initial_block: Set to True to subsample the initial convolution and
                add a MaxPool2D before the dense blocks are added.
        subsample_initial:
        activation: Type of activation at the top layer. Can be one of 'softmax' or 'sigmoid'.
                Note that if sigmoid is used, classes must be 1.
    Returns: keras tensor with nb_layers of conv_block appended
    '''

    concat_axis = 1 if K.image_data_format() == 'channels_first' else -1

    if reduction != 0.0:
        assert reduction <= 1.0 and reduction > 0.0, 'reduction value must lie between 0.0 and 1.0'

    # layers in each dense block
    if type(nb_layers_per_block) is list or type(nb_layers_per_block) is tuple:
        nb_layers = list(nb_layers_per_block)  # Convert tuple to list

        assert len(nb_layers) == (nb_dense_block), 'If list, nb_layer is used as provided. ' \
                                                   'Note that list size must be (nb_dense_block)'
        final_nb_layer = nb_layers[-1]
        nb_layers = nb_layers[:-1]
    else:
        if nb_layers_per_block == -1:
            assert (depth - 4) % 3 == 0, 'Depth must be 3 N + 4 if nb_layers_per_block == -1'
            count = int((depth - 4) / 3)

            if bottleneck:
                count = count // 2

            nb_layers = [count for _ in range(nb_dense_block)]
            final_nb_layer = count
        else:
            final_nb_layer = nb_layers_per_block
            nb_layers = [nb_layers_per_block] * nb_dense_block

    # compute initial nb_filter if -1, else accept users initial nb_filter
    if nb_filter <= 0:
        nb_filter = 2 * growth_rate

    # compute compression factor
    compression = 1.0 - reduction

    # Initial convolution
    if subsample_initial_block:
        initial_kernel = (7, 7)
        initial_strides = (2, 2)
    else:
        initial_kernel = (3, 3)
        initial_strides = (1, 1)

    x = Conv2D(nb_filter, initial_kernel, kernel_initializer='he_normal', padding='same',
               strides=initial_strides, use_bias=False, kernel_regularizer=l2(weight_decay))(img_input)

    if subsample_initial_block:
        x = BatchNormalization(axis=concat_axis, epsilon=1.1e-5)(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = __dense_block(x, nb_layers[block_idx], nb_filter, growth_rate, bottleneck=bottleneck,
                                     dropout_rate=dropout_rate, weight_decay=weight_decay)
        # add transition_block
        x = __transition_block(x, nb_filter, compression=compression, weight_decay=weight_decay)
        nb_filter = int(nb_filter * compression)

    # The last dense_block does not have a transition_block
    x, nb_filter = __dense_block(x, final_nb_layer, nb_filter, growth_rate, bottleneck=bottleneck,
                                 dropout_rate=dropout_rate, weight_decay=weight_decay)

    x = BatchNormalization(axis=concat_axis, epsilon=1.1e-5)(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)

    if include_top:
        x = Dense(nb_classes, activation=activation)(x)

    return x
  

def __transition_block(ip, nb_filter, compression=1.0, weight_decay=1e-4):
    ''' Apply BatchNorm, Relu 1x1, Conv2D, optional compression, dropout and Maxpooling2D
    Args:
        ip: keras tensor
        nb_filter: number of filters
        compression: calculated as 1 - reduction. Reduces the number of feature maps
                    in the transition block.
        dropout_rate: dropout rate
        weight_decay: weight decay factor
    Returns: keras tensor, after applying batch_norm, relu-conv, dropout, maxpool
    '''
    concat_axis = 1 if K.image_data_format() == 'channels_first' else -1

    x = BatchNormalization(axis=concat_axis, epsilon=1.1e-5)(ip)
    x = Activation('relu')(x)
    x = Conv2D(int(nb_filter * compression), (1, 1), kernel_initializer='he_normal', padding='same', use_bias=False,
               kernel_regularizer=l2(weight_decay))(x)
    x = AveragePooling2D((2, 2), strides=(2, 2))(x)

    return x

def __dense_block(x, nb_layers, nb_filter, growth_rate, bottleneck=False, dropout_rate=None, weight_decay=1e-4,
                  grow_nb_filters=True, return_concat_list=False):
    ''' Build a dense_block where the output of each conv_block is fed to subsequent ones
    Args:
        x: keras tensor
        nb_layers: the number of layers of conv_block to append to the model.
        nb_filter: number of filters
        growth_rate: growth rate
        bottleneck: bottleneck block
        dropout_rate: dropout rate
        weight_decay: weight decay factor
        grow_nb_filters: flag to decide to allow number of filters to grow
        return_concat_list: return the list of feature maps along with the actual output
    Returns: keras tensor with nb_layers of conv_block appended
    '''
    concat_axis = 1 if K.image_data_format() == 'channels_first' else -1

    x_list = [x]

    for i in range(nb_layers):
        cb = __conv_block(x, growth_rate, bottleneck, dropout_rate, weight_decay)
        x_list.append(cb)

        x = concatenate([x, cb], axis=concat_axis)

        if grow_nb_filters:
            nb_filter += growth_rate

    if return_concat_list:
        return x, nb_filter, x_list
    else:
        return x, nb_filter

def __conv_block(ip, nb_filter, bottleneck=False, dropout_rate=None, weight_decay=1e-4):
  ''' Apply BatchNorm, Relu, 3x3 Conv2D, optional bottleneck block and dropout
  Args:
      ip: Input keras tensor
      nb_filter: number of filters
      bottleneck: add bottleneck block
      dropout_rate: dropout rate
      weight_decay: weight decay factor
  Returns: keras tensor with batch_norm, relu and convolution2d added (optional bottleneck)
  '''
  concat_axis = 1 if K.image_data_format() == 'channels_first' else -1

  x = BatchNormalization(axis=concat_axis, epsilon=1.1e-5)(ip)
  x = Activation('relu')(x)

  if bottleneck:
      inter_channel = nb_filter * 4  # Obtained from https://github.com/liuzhuang13/DenseNet/blob/master/densenet.lua

      x = Conv2D(inter_channel, (1, 1), kernel_initializer='he_normal', padding='same', use_bias=False,
                 kernel_regularizer=l2(weight_decay))(x)
      x = BatchNormalization(axis=concat_axis, epsilon=1.1e-5)(x)
      x = Activation('relu')(x)

  x = Conv2D(nb_filter, (3, 3), kernel_initializer='he_normal', padding='same', use_bias=False)(x)
  if dropout_rate:
      x = Dropout(dropout_rate)(x)

  return x

def DenseNet(input_shape=None, depth=40, nb_dense_block=3, growth_rate=12, nb_filter=-1, nb_layers_per_block=-1,
             bottleneck=False, reduction=0.0, dropout_rate=0.0, weight_decay=1e-4, subsample_initial_block=False,
             include_top=True, weights=None, input_tensor=None,
             classes=10, activation='softmax'):
    '''Instantiate the DenseNet architecture,
        optionally loading weights pre-trained
        on CIFAR-10. Note that when using TensorFlow,
        for best performance you should set
        `image_data_format='channels_last'` in your Keras config
        at ~/.keras/keras.json.
        The model and the weights are compatible with both
        TensorFlow and Theano. The dimension ordering
        convention used by the model is the one
        specified in your Keras config file.
        # Arguments
            input_shape: optional shape tuple, only to be specified
                if `include_top` is False (otherwise the input shape
                has to be `(32, 32, 3)` (with `channels_last` dim ordering)
                or `(3, 32, 32)` (with `channels_first` dim ordering).
                It should have exactly 3 inputs channels,
                and width and height should be no smaller than 8.
                E.g. `(200, 200, 3)` would be one valid value.
            depth: number or layers in the DenseNet
            nb_dense_block: number of dense blocks to add to end (generally = 3)
            growth_rate: number of filters to add per dense block
            nb_filter: initial number of filters. -1 indicates initial
                number of filters is 2 * growth_rate
            nb_layers_per_block: number of layers in each dense block.
                Can be a -1, positive integer or a list.
                If -1, calculates nb_layer_per_block from the network depth.
                If positive integer, a set number of layers per dense block.
                If list, nb_layer is used as provided. Note that list size must
                be (nb_dense_block + 1)
            bottleneck: flag to add bottleneck blocks in between dense blocks
            reduction: reduction factor of transition blocks.
                Note : reduction value is inverted to compute compression.
            dropout_rate: dropout rate
            weight_decay: weight decay rate
            subsample_initial_block: Set to True to subsample the initial convolution and
                add a MaxPool2D before the dense blocks are added.
            include_top: whether to include the fully-connected
                layer at the top of the network.
            weights: one of `None` (random initialization) or
                'imagenet' (pre-training on ImageNet)..
            input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
                to use as image input for the model.
            classes: optional number of classes to classify images
                into, only to be specified if `include_top` is True, and
                if no `weights` argument is specified.
            activation: Type of activation at the top layer. Can be one of 'softmax' or 'sigmoid'.
                Note that if sigmoid is used, classes must be 1.
        # Returns
            A Keras model instance.
        '''

    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `cifar10` '
                         '(pre-training on CIFAR-10).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as ImageNet with `include_top`'
                         ' as true, `classes` should be 1000')

    if activation not in ['softmax', 'sigmoid']:
        raise ValueError('activation must be one of "softmax" or "sigmoid"')

    if activation == 'sigmoid' and classes != 1:
        raise ValueError('sigmoid activation can only be used when classes = 1')

    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=32,
                                      min_size=8,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    x = __create_dense_net(classes, img_input, include_top, depth, nb_dense_block,
                           growth_rate, nb_filter, nb_layers_per_block, bottleneck, reduction,
                           dropout_rate, weight_decay, subsample_initial_block, activation)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='densenet')

    # load weights
    if weights == 'imagenet':
        weights_loaded = False

        if (depth == 121) and (nb_dense_block == 4) and (growth_rate == 32) and (nb_filter == 64) and \
                (bottleneck is True) and (reduction == 0.5) and (dropout_rate == 0.0) and (subsample_initial_block):
            if include_top:
                weights_path = get_file('DenseNet-BC-121-32.h5',
                                        DENSENET_121_WEIGHTS_PATH,
                                        cache_subdir='models',
                                        md5_hash='a439dd41aa672aef6daba4ee1fd54abd')
            else:
                weights_path = get_file('DenseNet-BC-121-32-no-top.h5',
                                        DENSENET_121_WEIGHTS_PATH_NO_TOP,
                                        cache_subdir='models',
                                        md5_hash='55e62a6358af8a0af0eedf399b5aea99')
            model.load_weights(weights_path)
            weights_loaded = True

        if (depth == 161) and (nb_dense_block == 4) and (growth_rate == 48) and (nb_filter == 96) and \
                (bottleneck is True) and (reduction == 0.5) and (dropout_rate == 0.0) and (subsample_initial_block):
            if include_top:
                weights_path = get_file('DenseNet-BC-161-48.h5',
                                        DENSENET_161_WEIGHTS_PATH,
                                        cache_subdir='models',
                                        md5_hash='6c326cf4fbdb57d31eff04333a23fcca')
            else:
                weights_path = get_file('DenseNet-BC-161-48-no-top.h5',
                                        DENSENET_161_WEIGHTS_PATH_NO_TOP,
                                        cache_subdir='models',
                                        md5_hash='1a9476b79f6b7673acaa2769e6427b92')
            model.load_weights(weights_path)
            weights_loaded = True

        if (depth == 169) and (nb_dense_block == 4) and (growth_rate == 32) and (nb_filter == 64) and \
                (bottleneck is True) and (reduction == 0.5) and (dropout_rate == 0.0) and (subsample_initial_block):
            if include_top:
                weights_path = get_file('DenseNet-BC-169-32.h5',
                                        DENSENET_169_WEIGHTS_PATH,
                                        cache_subdir='models',
                                        md5_hash='914869c361303d2e39dec640b4e606a6')
            else:
                weights_path = get_file('DenseNet-BC-169-32-no-top.h5',
                                        DENSENET_169_WEIGHTS_PATH_NO_TOP,
                                        cache_subdir='models',
                                        md5_hash='89c19e8276cfd10585d5fadc1df6859e')
            model.load_weights(weights_path)
            weights_loaded = True

        if weights_loaded:
            if K.backend() == 'theano':
                convert_all_kernels_in_model(model)

            if K.image_data_format() == 'channels_first' and K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')

            print("Weights for the model were loaded successfully")

    return model


def DenseNetImageNet121(input_shape=None,
                        bottleneck=True,
                        reduction=0.5,
                        dropout_rate=0.0,
                        weight_decay=1e-4,
                        include_top=True,
                        weights='imagenet',
                        input_tensor=None,
                        classes=1000,
                        activation='softmax'):
    return DenseNet(input_shape, depth=121, nb_dense_block=4, growth_rate=32, nb_filter=64,
                    nb_layers_per_block=[6, 12, 24, 16], bottleneck=bottleneck, reduction=reduction,
                    dropout_rate=dropout_rate, weight_decay=weight_decay, subsample_initial_block=True,
                    include_top=include_top, weights=weights, input_tensor=input_tensor,
                    classes=classes, activation=activation)


/home/paperspace/anaconda3/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [0]:
import keras
import numpy as np
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions

model = keras.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

#x = preprocess_input(x)

preds = model.predict(x)

print('Predicted:', decode_predictions(preds))

('Predicted:', [[(u'n03180011', u'desktop_computer', 0.3106413), (u'n06359193', u'web_site', 0.23376717), (u'n03249569', u'drum', 0.1802558), (u'n04380533', u'table_lamp', 0.16848156), (u'n02105251', u'briard', 0.042605225)]])


In [0]:
import numpy as np
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
size = 224

model = DenseNetImageNet121(input_shape=(size, size, 3))
model.summary()


img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(size, size))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = preprocess_input(x)

preds = model.predict(x)

print('Predicted:', decode_predictions(preds))

Weights for the model were loaded successfully
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_121 (Conv2D)             (None, 112, 112, 64) 9408        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_122 (BatchN (None, 112, 112, 64) 256         conv2d_121[0][0]                 
__________________________________________________________________________________________________
activation_122 (Activation)     (None, 112, 112, 64) 0           batch_normalization_122[0][0]    
______________________________________________________________

__________________________________________________________________________________________________
activation_169 (Activation)     (None, 14, 14, 384)  0           batch_normalization_169[0][0]    
__________________________________________________________________________________________________
conv2d_168 (Conv2D)             (None, 14, 14, 128)  49152       activation_169[0][0]             
__________________________________________________________________________________________________
batch_normalization_170 (BatchN (None, 14, 14, 128)  512         conv2d_168[0][0]                 
__________________________________________________________________________________________________
activation_170 (Activation)     (None, 14, 14, 128)  0           batch_normalization_170[0][0]    
__________________________________________________________________________________________________
conv2d_169 (Conv2D)             (None, 14, 14, 32)   36864       activation_170[0][0]             
__________

In [0]:
import pickle as pickle
import numpy as np
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

counter = 0

def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    for line in doc.split('\n'):
        if len(line) < 1:
            continue
        #identifier = line.split('.')[0]
        dataset.append(line)
    return set(dataset)
    
def load_image(path):
	img = image.load_img(path, target_size=(224,224))
	x = image.img_to_array(img)
	# I commented the below line and added the one below that
	x = np.expand_dims(x, axis=0)
	# x = x.reshape((1, x.shape[0], x.shape[1], x.shape[2]))
	x = preprocess_input(x)
	return np.asarray(x)

def load_encoding_model():
    size = 224
    #model = DenseNetImageNet121(input_shape=(size, size, 3), include_top=False)
    model = DenseNetImageNet121(input_shape=(size, size, 3))
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    for layer in model.layers:
        layer.trainable = False
    return model

def get_encoding(model, img):
	global counter
	counter += 1
	image = load_image('Flickr8k_Dataset/'+str(img))
	pred = model.predict(image)
	# This should be done only if include_top is True
	pred = np.reshape(pred, pred.shape[1])
	if counter%1000 ==0: 
		print ("Encoding image: "+str(counter))
		print (pred.shape)
	return pred

def prepare_dataset(no_imgs = -1):
	f_train_images = open('Flickr8k_text/Flickr_8k.trainImages.txt','rb')
	train_imgs = f_train_images.read().strip().split('\n') if no_imgs == -1 else f_train_images.read().strip().split('\n')[:no_imgs]
	f_train_images.close()

	#train_imgs = load_set('Flickr8k_text/Flickr_8k.trainImages.txt')
    
	f_test_images = open('Flickr8k_text/Flickr_8k.testImages.txt','rb')
	test_imgs = f_test_images.read().strip().split('\n') if no_imgs == -1 else f_test_images.read().strip().split('\n')[:no_imgs]
	f_test_images.close()

	f_train_dataset = open('Flickr8k_text/flickr_8k_train_dataset.txt','wb')
	f_train_dataset.write("image_id\tcaptions\n")

	f_test_dataset = open('Flickr8k_text/flickr_8k_test_dataset.txt','wb')
	f_test_dataset.write("image_id\tcaptions\n")

	f_captions = open('Flickr8k_text/Flickr8k.token.txt', 'rb')
	captions = f_captions.read().strip().split('\n')
	data = {}
	for row in captions:
		row = row.split("\t")
		row[0] = row[0][:len(row[0])-2]
		try:
			data[row[0]].append(row[1])
		except:
			data[row[0]] = [row[1]]
	f_captions.close()

	encoded_images = {}
	encoding_model = load_encoding_model()

	c_train = 0
	for img in train_imgs:
		#print ("Encoding image: "+str(img))
		encoded_images[img] = get_encoding(encoding_model, img)
		for capt in data[img]:
			caption = "<start> "+capt+" <end>"
			f_train_dataset.write(img+"\t"+caption+"\n")
			f_train_dataset.flush()
			c_train += 1
	f_train_dataset.close()

	c_test = 0
	for img in test_imgs:
		encoded_images[img] = get_encoding(encoding_model, img)
		for capt in data[img]:
			caption = "<start> "+capt+" <end>"
			f_test_dataset.write(img+"\t"+caption+"\n")
			f_test_dataset.flush()
			c_test += 1
	f_test_dataset.close()
	with open( "encoded_images.p", "wb" ) as pickle_f:
		pickle.dump( encoded_images, pickle_f )
	return [c_train, c_test]

In [0]:
c_train, c_test = prepare_dataset()
print ("Training samples = "+str(c_train))
print ("Test samples = "+str(c_test))

Weights for the model were loaded successfully
Encoding image: 1000
(1024,)
Encoding image: 2000
(1024,)
Encoding image: 3000
(1024,)
Encoding image: 4000
(1024,)
Encoding image: 5000
(1024,)
Encoding image: 6000
(1024,)
Encoding image: 7000
(1024,)
Training samples = 30000
Test samples = 5000


In [0]:
from keras.applications import inception_v3
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Merge, Activation, Flatten
from keras.preprocessing import image, sequence
from keras.callbacks import ModelCheckpoint
import cPickle as pickle

EMBEDDING_DIM = 128


class CaptionGenerator():

    def __init__(self):
        self.max_cap_len = None
        self.vocab_size = None
        self.index_word = None
        self.word_index = None
        self.total_samples = None
        self.encoded_images = pickle.load( open( "encoded_images.p", "rb" ) )
        self.variable_initializer()

    def variable_initializer(self):
        df = pd.read_csv('Flickr8k_text/flickr_8k_train_dataset.txt', delimiter='\t')
        nb_samples = df.shape[0]
        iter = df.iterrows()
        caps = []
        for i in range(nb_samples):
            x = iter.next()
            caps.append(x[1][1])

        self.total_samples=0
        for text in caps:
            self.total_samples+=len(text.split())-1
        print ("Total samples : " + str(self.total_samples))
        
        words = [txt.split() for txt in caps]
        unique = []
        for word in words:
            unique.extend(word)

        unique = list(set(unique))
        self.vocab_size = len(unique)
        self.word_index = {}
        self.index_word = {}
        for i, word in enumerate(unique):
            self.word_index[word]=i
            self.index_word[i]=word

        max_len = 0
        for caption in caps:
            if(len(caption.split()) > max_len):
                max_len = len(caption.split())
        self.max_cap_len = max_len
        print ("Vocabulary size: "+str(self.vocab_size))
        print ("Maximum caption length: "+str(self.max_cap_len))
        print ("Variables initialization done!")


    def data_generator(self, batch_size = 32):
        partial_caps = []
        next_words = []
        images = []
        print ("Generating data...")
        gen_count = 0
        df = pd.read_csv('Flickr8k_text/flickr_8k_train_dataset.txt', delimiter='\t')
        nb_samples = df.shape[0]
        iter = df.iterrows()
        caps = []
        imgs = []
        for i in range(nb_samples):
            x = iter.next()
            caps.append(x[1][1])
            imgs.append(x[1][0])


        total_count = 0
        while 1:
            image_counter = -1
            for text in caps:
                image_counter+=1
                current_image = self.encoded_images[imgs[image_counter]]
                for i in range(len(text.split())-1):
                    total_count+=1
                    partial = [self.word_index[txt] for txt in text.split()[:i+1]]
                    partial_caps.append(partial)
                    next = np.zeros(self.vocab_size)
                    next[self.word_index[text.split()[i+1]]] = 1
                    next_words.append(next)
                    images.append(current_image)

                    if total_count>=batch_size:
                        next_words = np.asarray(next_words)
                        images = np.asarray(images)
                        partial_caps = sequence.pad_sequences(partial_caps, maxlen=self.max_cap_len, padding='post')
                        total_count = 0
                        gen_count+=1
                        if gen_count%1000 ==0: print ("yielding count: "+str(gen_count))
                        yield [[images, partial_caps], next_words]
                        partial_caps = []
                        next_words = []
                        images = []
        
    def load_image(self, path):
        img = image.load_img(path, target_size=(224,224))
        x = image.img_to_array(img)
        return np.asarray(x)


    def create_model(self, ret_model = False):
        #base_model = VGG16(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
        #base_model.trainable=False
        image_model = Sequential()
        #image_model.add(base_model)
        #image_model.add(Flatten())
        image_model.add(Dense(EMBEDDING_DIM, input_dim = 1024, activation='relu'))

        image_model.add(RepeatVector(self.max_cap_len))

        lang_model = Sequential()
        lang_model.add(Embedding(self.vocab_size, 256, input_length=self.max_cap_len))
        lang_model.add(LSTM(256,return_sequences=True))
        lang_model.add(TimeDistributed(Dense(EMBEDDING_DIM)))

        model = Sequential()
        model.add(Merge([image_model, lang_model], mode='concat'))
        model.add(LSTM(1000,return_sequences=False))
        model.add(Dense(self.vocab_size))
        model.add(Activation('softmax'))

        print ("Model created!")

        if(ret_model==True):
            return model

        model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
        return model

    def get_word(self,index):
        return self.index_word[index]

/home/paperspace/anaconda3/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [0]:
import keras
from keras.callbacks import ModelCheckpoint

def train_model(weight = None, batch_size=32, epochs = 10, initial_epoch = 0):

    cg = CaptionGenerator()
    model = cg.create_model()

    if weight != None:
        model.load_weights(weight)

    #counter = 0
    #file_name = 'weights-improvement-{epoch:02d}.hdf5'
    #checkpoint = ModelCheckpoint(file_name, monitor='loss', verbose=1, save_best_only=True, mode='min')
    #callbacks_list = [checkpoint]
    #model.fit_generator(cg.data_generator(batch_size=batch_size), steps_per_epoch=cg.total_samples/batch_size, epochs=epochs, verbose=2, callbacks=callbacks_list)
    
       
    counter = 0
    fileid='RMSProp'
    logfilename = fileid + "-log.csv"
    csv_logger = keras.callbacks.CSVLogger(logfilename, separator=',', append=True)

    file_name = 'weights-improvement-{epoch:02d}.hdf5'
    checkpoint = ModelCheckpoint(file_name, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint, csv_logger]
    model.fit_generator(cg.data_generator(batch_size=batch_size), initial_epoch=initial_epoch, steps_per_epoch=cg.total_samples/batch_size, epochs=epochs, verbose=1, callbacks=callbacks_list)
    
    
    try:
        model.save('Models/WholeModel.h5', overwrite=True)
        model.save_weights('Models/Weights.h5',overwrite=True)
    except:
        print ("Error in saving model.")
    print ("Training complete...\n")

In [0]:
import time
tic = time.clock()
train_model(initial_epoch = 0, epochs=1, batch_size=2176)
toc = time.clock()
print('Time taken to train with 50 epochs is ', ((toc - tic)/60))

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Variables initialization done!


/home/paperspace/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:127: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Model created!
Generating data...
Epoch 1/1
177/176 [==============================] - 292s 2s/step - loss: 5.3625 - acc: 0.1029

Epoch 00001: loss improved from inf to 5.36252, saving model to weights-improvement-01.hdf5
Error in saving model.
Training complete...

Time taken to train with 50 epochs is  3.8095816


In [0]:
import time
tic = time.clock()
train_model(initial_epoch = 1, epochs=5, batch_size=2176, weight='weights-improvement-01.hdf5')
toc = time.clock()
print('Time taken to train with 50 epochs is ', ((toc - tic)/60))

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Variables initialization done!


/home/paperspace/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:127: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Model created!
Epoch 2/5
Generating data...
177/176 [==============================] - 294s 2s/step - loss: 5.1209 - acc: 0.1311

Epoch 00002: loss improved from inf to 5.12086, saving model to weights-improvement-02.hdf5
Epoch 3/5
177/176 [==============================] - 275s 2s/step - loss: 4.6336 - acc: 0.2213

Epoch 00003: loss improved from 5.12086 to 4.63364, saving model to weights-improvement-03.hdf5
Epoch 4/5
177/176 [==============================] - 275s 2s/step - loss: 4.4040 - acc: 0.2446

Epoch 00004: loss improved from 4.63364 to 4.40397, saving model to weights-improvement-04.hdf5
Epoch 5/5
177/176 [==============================] - 275s 2s/step - loss: 4.2418 - acc: 0.2611

Epoch 00005: loss improved from 4.40397 to 4.24180, saving model to weights-improvement-05.hdf5
Error in saving model.
Training complete...

Time taken to train with 50 epochs is  13.41220605


In [0]:
import time
tic = time.clock()
train_model(initial_epoch = 5, epochs=10, batch_size=2176, weight='weights-improvement-05.hdf5')
toc = time.clock()
print('Time taken to train with 5 epochs is ', ((toc - tic)/60))

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Variables initialization done!


/home/paperspace/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:127: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Model created!
Epoch 6/10Generating data...

176/176 [==============================] - 248s 1s/step - loss: 4.0713 - acc: 0.2789

Epoch 00006: loss improved from inf to 4.07130, saving model to weights-improvement-06.hdf5
Epoch 7/10
176/176 [==============================] - 238s 1s/step - loss: 3.7055 - acc: 0.3165

Epoch 00007: loss improved from 4.07130 to 3.70549, saving model to weights-improvement-07.hdf5
Epoch 8/10
176/176 [==============================] - 239s 1s/step - loss: 3.4561 - acc: 0.3393

Epoch 00008: loss improved from 3.70549 to 3.45613, saving model to weights-improvement-08.hdf5
Epoch 9/10
176/176 [==============================] - 239s 1s/step - loss: 3.2773 - acc: 0.3552

Epoch 00009: loss improved from 3.45613 to 3.27727, saving model to weights-improvement-09.hdf5
Epoch 10/10
176/176 [==============================] - 239s 1s/step - loss: 3.1361 - acc: 0.3682

Epoch 00010: loss improved from 3.27727 to 3.13608, saving model to weights-improvement-10.hdf5
Erro

In [0]:
import time
tic = time.clock()
train_model(initial_epoch = 10, epochs=15, batch_size=2176, weight='weights-improvement-10.hdf5')
toc = time.clock()
print("Time taken to train with 5 epochs is " + str(((toc - tic)/60)) + " minutes")

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Variables initialization done!


/home/paperspace/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:127: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Model created!
Generating data...
Epoch 11/15
176/176 [==============================] - 249s 1s/step - loss: 3.0628 - acc: 0.3770

Epoch 00011: loss improved from inf to 3.06276, saving model to weights-improvement-11.hdf5
Epoch 12/15
176/176 [==============================] - 241s 1s/step - loss: 2.9291 - acc: 0.3869

Epoch 00012: loss improved from 3.06276 to 2.92912, saving model to weights-improvement-12.hdf5
Epoch 13/15
176/176 [==============================] - 241s 1s/step - loss: 2.8296 - acc: 0.3958

Epoch 00013: loss improved from 2.92912 to 2.82963, saving model to weights-improvement-13.hdf5
Epoch 14/15
176/176 [==============================] - 241s 1s/step - loss: 2.7429 - acc: 0.4036

Epoch 00014: loss improved from 2.82963 to 2.74286, saving model to weights-improvement-14.hdf5
Epoch 15/15
176/176 [==============================] - 241s 1s/step - loss: 2.6607 - acc: 0.4110

Epoch 00015: loss improved from 2.74286 to 2.66072, saving model to weights-improvement-15.hdf5


In [0]:
import time
tic = time.clock()
train_model(initial_epoch = 15, epochs=20, batch_size=2176, weight='weights-improvement-15.hdf5')
toc = time.clock()
print("Time taken to train with 5 epochs is " + str(((toc - tic)/60)) + " minutes")

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Variables initialization done!


/home/paperspace/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:127: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Model created!
Epoch 16/20
Generating data...
176/176 [==============================] - 254s 1s/step - loss: 2.6336 - acc: 0.4155

Epoch 00016: loss improved from inf to 2.63358, saving model to weights-improvement-16.hdf5
Epoch 17/20
176/176 [==============================] - 246s 1s/step - loss: 2.5241 - acc: 0.4260

Epoch 00017: loss improved from 2.63358 to 2.52411, saving model to weights-improvement-17.hdf5
Epoch 18/20
176/176 [==============================] - 247s 1s/step - loss: 2.4518 - acc: 0.4333

Epoch 00018: loss improved from 2.52411 to 2.45180, saving model to weights-improvement-18.hdf5
Epoch 19/20
176/176 [==============================] - 246s 1s/step - loss: 2.3836 - acc: 0.4422

Epoch 00019: loss improved from 2.45180 to 2.38357, saving model to weights-improvement-19.hdf5
Epoch 20/20
176/176 [==============================] - 246s 1s/step - loss: 2.3059 - acc: 0.4518

Epoch 00020: loss improved from 2.38357 to 2.30587, saving model to weights-improvement-20.hdf5


In [0]:
tic = time.clock()
train_model(initial_epoch = 20, epochs=25, batch_size=2176, weight='weights-improvement-20.hdf5')
toc = time.clock()
print("Time taken to train with 5 epochs is " + str(((toc - tic)/60)) + " minutes")

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Variables initialization done!


/home/paperspace/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:127: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Model created!
Generating data...
Epoch 21/25
176/176 [==============================] - 262s 1s/step - loss: 2.3078 - acc: 0.4529

Epoch 00021: loss improved from inf to 2.30783, saving model to weights-improvement-21.hdf5
Epoch 22/25
176/176 [==============================] - 253s 1s/step - loss: 2.2081 - acc: 0.4667

Epoch 00022: loss improved from 2.30783 to 2.20812, saving model to weights-improvement-22.hdf5
Epoch 23/25
101/176 [================>.............] - ETA: 1:47 - loss: 2.1444 - acc: 0.4743

In [0]:
import time
tic = time.clock()
train_model(initial_epoch = 22, epochs=25, batch_size=2176, weight='weights-improvement-22.hdf5')
toc = time.clock()
print("Time taken to train with 3 epochs is " + str(((toc - tic)/60)) + " minutes")

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Variables initialization done!


/home/paperspace/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:127: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Model created!
Generating data...
Epoch 23/25
176/176 [==============================] - 245s 1s/step - loss: 2.2045 - acc: 0.4679

Epoch 00023: loss improved from inf to 2.20453, saving model to weights-improvement-23.hdf5
Epoch 24/25
176/176 [==============================] - 240s 1s/step - loss: 2.1105 - acc: 0.4809

Epoch 00024: loss improved from 2.20453 to 2.11053, saving model to weights-improvement-24.hdf5
Epoch 25/25
176/176 [==============================] - 241s 1s/step - loss: 2.0498 - acc: 0.4892

Epoch 00025: loss improved from 2.11053 to 2.04985, saving model to weights-improvement-25.hdf5
Error in saving model.
Training complete...

Time taken to train with 3 epochs is 9.67986563333 minutes


In [0]:
import time
tic = time.clock()
initial_epoch = 25
epochs=30

train_model(initial_epoch = initial_epoch, epochs=epochs, batch_size=2176, weight='weights-improvement-25.hdf5')

toc = time.clock()
print("Time taken to train with " + str(epochs - initial_epoch) + " epochs is " + str(((toc - tic)/60)) + " minutes")

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Variables initialization done!


/home/paperspace/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:127: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Model created!
Generating data...
Epoch 26/30
176/176 [==============================] - 254s 1s/step - loss: 2.0434 - acc: 0.4924

Epoch 00026: loss improved from inf to 2.04342, saving model to weights-improvement-26.hdf5
Epoch 27/30
176/176 [==============================] - 251s 1s/step - loss: 1.9533 - acc: 0.5062

Epoch 00027: loss improved from 2.04342 to 1.95334, saving model to weights-improvement-27.hdf5
Epoch 28/30
176/176 [==============================] - 251s 1s/step - loss: 1.9042 - acc: 0.5140

Epoch 00028: loss improved from 1.95334 to 1.90417, saving model to weights-improvement-28.hdf5
Epoch 29/30
176/176 [==============================] - 251s 1s/step - loss: 1.8523 - acc: 0.5226

Epoch 00029: loss improved from 1.90417 to 1.85231, saving model to weights-improvement-29.hdf5
Epoch 30/30
176/176 [==============================] - 251s 1s/step - loss: 1.8054 - acc: 0.5310

Epoch 00030: loss improved from 1.85231 to 1.80538, saving model to weights-improvement-30.hdf5


In [0]:
import time
tic = time.clock()
initial_epoch = 30
epochs=35
weight = 'weights-improvement-30.hdf5'

train_model(initial_epoch = initial_epoch, epochs=epochs, batch_size=2176, weight=weight)

toc = time.clock()
print("Time taken to train with " + str(epochs - initial_epoch) + " epochs is " + str(((toc - tic)/60)) + " minutes")

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Variables initialization done!


/home/paperspace/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:127: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Model created!
Generating data...
Epoch 31/35
176/176 [==============================] - 256s 1s/step - loss: 1.8103 - acc: 0.5309

Epoch 00031: loss improved from inf to 1.81031, saving model to weights-improvement-31.hdf5
Epoch 32/35
176/176 [==============================] - 253s 1s/step - loss: 1.7219 - acc: 0.5460

Epoch 00032: loss improved from 1.81031 to 1.72194, saving model to weights-improvement-32.hdf5
Epoch 33/35
176/176 [==============================] - 252s 1s/step - loss: 1.6755 - acc: 0.5547

Epoch 00033: loss improved from 1.72194 to 1.67546, saving model to weights-improvement-33.hdf5
Epoch 34/35
176/176 [==============================] - 252s 1s/step - loss: 1.6288 - acc: 0.5640

Epoch 00034: loss improved from 1.67546 to 1.62877, saving model to weights-improvement-34.hdf5
Epoch 35/35
176/176 [==============================] - 254s 1s/step - loss: 1.5853 - acc: 0.5731

Epoch 00035: loss improved from 1.62877 to 1.58527, saving model to weights-improvement-35.hdf5


In [0]:
import time
tic = time.clock()
initial_epoch = 35
epochs=40
weight = 'weights-improvement-35.hdf5'

train_model(initial_epoch = initial_epoch, epochs=epochs, batch_size=2176, weight=weight)

toc = time.clock()
print("Time taken to train with " + str(epochs - initial_epoch) + " epochs is " + str(((toc - tic)/60)) + " minutes")

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Variables initialization done!


/home/paperspace/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:127: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Model created!
Epoch 36/40Generating data...

176/176 [==============================] - 260s 1s/step - loss: 1.5983 - acc: 0.5708

Epoch 00036: loss improved from inf to 1.59833, saving model to weights-improvement-36.hdf5
Epoch 37/40
176/176 [==============================] - 256s 1s/step - loss: 1.5178 - acc: 0.5865

Epoch 00037: loss improved from 1.59833 to 1.51777, saving model to weights-improvement-37.hdf5
Epoch 38/40
176/176 [==============================] - 256s 1s/step - loss: 1.4757 - acc: 0.5945

Epoch 00038: loss improved from 1.51777 to 1.47569, saving model to weights-improvement-38.hdf5
Epoch 39/40
176/176 [==============================] - 257s 1s/step - loss: 1.4404 - acc: 0.6028

Epoch 00039: loss improved from 1.47569 to 1.44039, saving model to weights-improvement-39.hdf5
Epoch 40/40
176/176 [==============================] - 258s 1s/step - loss: 1.3984 - acc: 0.6112

Epoch 00040: loss improved from 1.44039 to 1.39839, saving model to weights-improvement-40.hdf5


In [0]:
import time
tic = time.clock()
initial_epoch = 40
epochs=45
weight = 'weights-improvement-40.hdf5'

train_model(initial_epoch = initial_epoch, epochs=epochs, batch_size=2176, weight=weight)

toc = time.clock()
print("Time taken to train with " + str(epochs - initial_epoch) + " epochs is " + str(((toc - tic)/60)) + " minutes")

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Variables initialization done!


/home/paperspace/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:127: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Model created!
Epoch 41/45Generating data...

176/176 [==============================] - 266s 2s/step - loss: 1.4208 - acc: 0.6071

Epoch 00041: loss improved from inf to 1.42083, saving model to weights-improvement-41.hdf5
Epoch 42/45
176/176 [==============================] - 260s 1s/step - loss: 1.3411 - acc: 0.6227

Epoch 00042: loss improved from 1.42083 to 1.34113, saving model to weights-improvement-42.hdf5
Epoch 43/45
176/176 [==============================] - 259s 1s/step - loss: 1.3093 - acc: 0.6303

Epoch 00043: loss improved from 1.34113 to 1.30933, saving model to weights-improvement-43.hdf5
Epoch 44/45
176/176 [==============================] - 259s 1s/step - loss: 1.2747 - acc: 0.6378

Epoch 00044: loss improved from 1.30933 to 1.27467, saving model to weights-improvement-44.hdf5
Epoch 45/45
176/176 [==============================] - 259s 1s/step - loss: 1.2463 - acc: 0.6442

Epoch 00045: loss improved from 1.27467 to 1.24628, saving model to weights-improvement-45.hdf5


In [0]:
import time
tic = time.clock()
initial_epoch = 45
epochs=50
weight = 'weights-improvement-45.hdf5'

train_model(initial_epoch = initial_epoch, epochs=epochs, batch_size=2176, weight=weight)

toc = time.clock()
print("Time taken to train with " + str(epochs - initial_epoch) + " epochs is " + str(((toc - tic)/60)) + " minutes")

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Variables initialization done!


/home/paperspace/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:127: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Model created!
Generating data...Epoch 46/50

176/176 [==============================] - 244s 1s/step - loss: 1.2672 - acc: 0.6395

Epoch 00046: loss improved from inf to 1.26718, saving model to weights-improvement-46.hdf5
Epoch 47/50
176/176 [==============================] - 241s 1s/step - loss: 1.1975 - acc: 0.6545

Epoch 00047: loss improved from 1.26718 to 1.19751, saving model to weights-improvement-47.hdf5
Epoch 48/50
176/176 [==============================] - 241s 1s/step - loss: 1.1685 - acc: 0.6620

Epoch 00048: loss improved from 1.19751 to 1.16852, saving model to weights-improvement-48.hdf5
Epoch 49/50
176/176 [==============================] - 243s 1s/step - loss: 1.1356 - acc: 0.6688

Epoch 00049: loss improved from 1.16852 to 1.13556, saving model to weights-improvement-49.hdf5
Epoch 50/50
176/176 [==============================] - 244s 1s/step - loss: 1.1090 - acc: 0.6755

Epoch 00050: loss improved from 1.13556 to 1.10903, saving model to weights-improvement-50.hdf5


In [0]:
import pickle as pickle
import numpy as np
from keras.preprocessing import sequence
import nltk

cg = CaptionGenerator()

def process_caption(caption):
	caption_split = caption.split()
	processed_caption = caption_split[1:]
	try:
		end_index = processed_caption.index('<end>')
		processed_caption = processed_caption[:end_index]
	except:
		pass
	return " ".join([word for word in processed_caption])

def get_best_caption(captions):
    captions.sort(key = lambda l:l[1])
    best_caption = captions[-1][0]
    return " ".join([cg.index_word[index] for index in best_caption])

def get_all_captions(captions):
    final_captions = []
    captions.sort(key = lambda l:l[1])
    for caption in captions:
        text_caption = " ".join([cg.index_word[index] for index in caption[0]])
        final_captions.append([text_caption, caption[1]])
    return final_captions

def generate_captions(model, image, beam_size):
	start = [cg.word_index['<start>']]
	captions = [[start,0.0]]
	while(len(captions[0][0]) < cg.max_cap_len):
		temp_captions = []
		for caption in captions:
			partial_caption = sequence.pad_sequences([caption[0]], maxlen=cg.max_cap_len, padding='post')
			next_words_pred = model.predict([np.asarray([image]), np.asarray(partial_caption)])[0]
			next_words = np.argsort(next_words_pred)[-beam_size:]
			for word in next_words:
				new_partial_caption, new_partial_caption_prob = caption[0][:], caption[1]
				new_partial_caption.append(word)
				new_partial_caption_prob+=next_words_pred[word]
				temp_captions.append([new_partial_caption,new_partial_caption_prob])
		captions = temp_captions
		captions.sort(key = lambda l:l[1])
		captions = captions[-beam_size:]

	return captions

def test_model(weight, img_name, beam_size = 3):
	encoded_images = pickle.load( open( "encoded_images.p", "rb" ) )
	model = cg.create_model(ret_model = True)
	model.load_weights(weight)

	image = encoded_images[img_name]
	captions = generate_captions(model, image, beam_size)
	return process_caption(get_best_caption(captions))
	#return [process_caption(caption[0]) for caption in get_all_captions(captions)] 

def bleu_score(hypotheses, references):
	return nltk.translate.bleu_score.corpus_bleu(references, hypotheses)

def test_model_on_images(weight, img_dir, beam_size = 3):
	imgs = []
	captions = {}
	with open(img_dir, 'rb') as f_images:
		imgs = f_images.read().strip().split('\n')
	encoded_images = pickle.load( open( "encoded_images.p", "rb" ) )
	model = cg.create_model(ret_model = True)
	model.load_weights(weight)

	f_pred_caption = open('predicted_captions.txt', 'wb')

	for count, img_name in enumerate(imgs):
		print ("Predicting for image: "+str(count))
		image = encoded_images[img_name]
		image_captions = generate_captions(model, image, beam_size)
		best_caption = process_caption(get_best_caption(image_captions))
		captions[img_name] = best_caption
		print (img_name+" : "+str(best_caption))
		f_pred_caption.write(img_name+"\t"+str(best_caption))
		f_pred_caption.flush()
	f_pred_caption.close()

	f_captions = open('Flickr8k_text/Flickr8k.token.txt', 'rb')
	captions_text = f_captions.read().strip().split('\n')
	image_captions_pair = {}
	for row in captions_text:
		row = row.split("\t")
		row[0] = row[0][:len(row[0])-2]
		try:
			image_captions_pair[row[0]].append(row[1])
		except:
			image_captions_pair[row[0]] = [row[1]]
	f_captions.close()
	
	hypotheses=[]
	references = []
	for img_name in imgs:
		hypothesis = captions[img_name]
		reference = image_captions_pair[img_name]
		hypotheses.append(hypothesis)
		references.append(reference)

	return bleu_score(hypotheses, references)


Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Variables initialization done!


In [0]:
weight = 'weights-improvement-50.hdf5'
#test_image = '3155451946_c0862c70cb.jpg'
test_img_dir = 'Flickr8k_text/Flickr_8k.testImages.txt'
#print test_model(weight, test_image)
print (test_model_on_images(weight, test_img_dir, beam_size=3))

/home/paperspace/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:127: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Model created!
Predicting for image: 0
3385593926_d3e9c21170.jpg : A black dog with the black snow covered background .
Predicting for image: 1
2677656448_6b7e7702af.jpg : A boy in a orange shirt is standing in a swimming pool .
Predicting for image: 2
311146855_0b65fdb169.jpg : A girl in a blue shirt is sitting on a blue slide .
Predicting for image: 3
1258913059_07c613f7ff.jpg : A man in a striped shirt and a striped shirt is holding a camera .
Predicting for image: 4
241347760_d44c8d3a01.jpg : A football player is wearing a red jersey .
Predicting for image: 5
2654514044_a70a6e2c21.jpg : A brown dog carries a tennis ball in its mouth .
Predicting for image: 6
2339106348_2df90aa6a9.jpg : Two young women are staring at the camera .
Predicting for image: 7
256085101_2c2617c5d0.jpg : A dog with a red collar in the snow .
Predicting for image: 8
280706862_14c30d734a.jpg : Two young white dogs are on a dirt path .
Predicting for image: 9
3072172967_630e9c69d0.jpg : A group of members and 

3119875880_22f9129a1c.jpg : a man wearing a blue shirt is sitting on a rock wall .
Predicting for image: 84
3339140382_2e49bc324a.jpg : A man is skiing down a ramp in the air .
Predicting for image: 85
2712787899_d85048eb6a.jpg : A man and a woman sit on a wood street .
Predicting for image: 86
3655155990_b0e201dd3c.jpg : A black and white dog jumps in the air .
Predicting for image: 87
3325497914_f9014d615b.jpg : A dog jumping over a hurdle .
Predicting for image: 88
468310111_d9396abcbd.jpg : A brown dog is running up in the grass .
Predicting for image: 89
747921928_48eb02aab2.jpg : a man wearing a red shirt and jeans .
Predicting for image: 90
3639967449_137f48b43d.jpg : A group of people are looking at a woman in a red coat .
Predicting for image: 91
2374652725_32f90fa15c.jpg : A black and white dog looking at the camera .
Predicting for image: 92
3363750526_efcedc47a9.jpg : A girl is holding a camera in a park .
Predicting for image: 93
2689001252_e0016c89f0.jpg : A boy in a blac

2300168895_a9b83e16fc.jpg : Two dogs playing in the grass .
Predicting for image: 167
106490881_5a2dd9b7bd.jpg : A young boy jumping in the snow .
Predicting for image: 168
3694991841_141804da1f.jpg : A dog jumps in the air to catch a ball .
Predicting for image: 169
1523984678_edd68464da.jpg : Two dogs are running on grass .
Predicting for image: 170
2529116152_4331dabf50.jpg : A man is performing a trick on a skateboard .
Predicting for image: 171
1773928579_5664a810dc.jpg : A group of people are playing soccer .
Predicting for image: 172
191003285_edd8d0cf58.jpg : A man and a dog are sitting on the edge of a building .
Predicting for image: 173
1392272228_cf104086e6.jpg : A black and white dog is playing in the water .
Predicting for image: 174
2910758605_73a3f5a5c2.jpg : A man and a woman sit on a cellphone .
Predicting for image: 175
3507076266_8b17993fbb.jpg : A child in a red helmet riding a dirt path .
Predicting for image: 176
535830521_aa971319fc.jpg : A group of people are s

3285993030_87b0f1d202.jpg : A black and white dog is chasing after a tennis ball .
Predicting for image: 251
3220161734_77f42734b9.jpg : A man wears a dark dark shirt .
Predicting for image: 252
2393264648_a280744f97.jpg : A group of kids watch a chunk of snow .
Predicting for image: 253
506367606_7cca2bba9b.jpg : A man and a woman wearing a black hat .
Predicting for image: 254
422763475_0bc814dac6.jpg : A black dog is on the ice of a ice .
Predicting for image: 255
1982852140_56425fa7a2.jpg : A group of people playing in the ocean .
Predicting for image: 256
2929506802_5432054d77.jpg : A brown dog is jumping into a tennis game .
Predicting for image: 257
541063517_35044c554a.jpg : A woman poses for a picture .
Predicting for image: 258
2595186208_9b16fa0ee3.jpg : A blond dog and a grey dog in a green field .
Predicting for image: 259
2922973230_5a769ef92a.jpg : A woman in a field in the green grass .
Predicting for image: 260
166507476_9be5b9852a.jpg : A man wearing a blue helmet is 

...

2196107384_361d73a170.jpg : A group of men are standing around the wall at night .
Predicting for image: 662
1561658940_a947f2446a.jpg : A little boy is jumping into the air .
Predicting for image: 663
3655074079_7df3812bc5.jpg : A woman in a white dress is playing a violin .
Predicting for image: 664
3004823335_9b82cbd8a7.jpg : A brown dog and a ball in the grass .
Predicting for image: 665
2495931537_9b8d4474b6.jpg : A group of people stand in a room seat .
Predicting for image: 666
293881927_ac62900fd4.jpg : A young girl in a green shirt is walking through the grass .
Predicting for image: 667
3162045919_c2decbb69b.jpg : A small black and white dog is jumping in the water .
Predicting for image: 668
505929313_7668f021ab.jpg : A little boy playing in the water .
Predicting for image: 669
3244470342_c08f6bb17e.jpg : A brown dog holds a red ball .
Predicting for image: 670
3655964639_21e76383d0.jpg : A little girl in a green shirt is playing on a park .
Predicting for image: 671
371896

434792818_56375e203f.jpg : A child in a red and red helmet climbing across the grass .
Predicting for image: 744
3635577874_48ebaac734.jpg : A man in a red shirt is skating .
Predicting for image: 745
3413571342_b9855795e2.jpg : The child is in the water next to the water .
Predicting for image: 746
2475162978_2c51048dca.jpg : A baseball player player player on the grass .
Predicting for image: 747
2160266952_a2ab39191b.jpg : A black dog is jumping over a red house .
Predicting for image: 748
463978865_c87c6ca84c.jpg : A group of men on a motorcycle are about to jump .
Predicting for image: 749
3585487286_ef9a8d4c56.jpg : A dog running in the snow .
Predicting for image: 750
3239021459_a6b71bb400.jpg : A man is standing in the middle of the ocean whilst holding a trick .
Predicting for image: 751
2662845514_8620aaee96.jpg : There is a kid wearing a red shirt playing tennis .
Predicting for image: 752
3044746136_8b89da5f40.jpg : A group of people are standing in front of a brick buildin

229862312_1a0ba19dab.jpg : A brown dog catching a ball in the grass .
Predicting for image: 825
3333921867_6cc7d7c73d.jpg : A large group of black dogs running in the grass .
Predicting for image: 826
2248487950_c62d0c81a9.jpg : A group of people in front of a red building .
Predicting for image: 827
3074842262_62b1b2168c.jpg : Two brown dogs are playing with a man .
Predicting for image: 828
561417861_8e25d0c0e8.jpg : A man in a red shirt and a striped shirt is standing on a ledge .
Predicting for image: 829
2646046871_c3a5dbb971.jpg : A brown dog is playing with a ball in his mouth .
Predicting for image: 830
2944952557_8484f0da8f.jpg : A boy in a red shirt and white shorts is about to kick the ball .
Predicting for image: 831
3244747165_17028936e0.jpg : Two women and a woman are posing for a picture .
Predicting for image: 832
2285570521_05015cbf4b.jpg : A man in a red shirt is jumping in the snow .
Predicting for image: 833
315880837_90db309bab.jpg : A brown and white dog in a red 

2934359101_cdf57442dc.jpg : A young boy wearing a black hat and white hat and black hat .
Predicting for image: 907
339658315_fbb178c252.jpg : A man in a brown shirt and white shirt is playing a game .
Predicting for image: 908
2991994415_504d1c0a03.jpg : A child in a red shirt is pointing at the bed .
Predicting for image: 909
3349451628_4249a21c8f.jpg : A black and white dog with a black toy in the air .
Predicting for image: 910
3197917064_e679a44b8e.jpg : A black dog plays in the woods .
Predicting for image: 911
2657484284_daa07a3a1b.jpg : A man in a red shirt and khaki shorts is hiking across a grassy field .
Predicting for image: 912
2894217628_f1a4153dca.jpg : A group of men are in white uniforms .
Predicting for image: 913
3424424006_98f9d1921c.jpg : The boy in the white t-shirt is airborne whilst being whilst wearing a helmet .
Predicting for image: 914
2526585002_10987a63f3.jpg : A boat in a lake .
Predicting for image: 915
1836335410_de8313a64e.jpg : A group of people in da

241345844_69e1c22464.jpg : A football player making a jersey .
Predicting for image: 991
3256043809_47258e0b3e.jpg : A black dog on the side of a red dog .
Predicting for image: 992
2351479551_e8820a1ff3.jpg : A man and a black and black dog in a tree .
Predicting for image: 993
3514179514_cbc3371b92.jpg : A brown dog and a brown dog play in a backyard .
Predicting for image: 994
1119015538_e8e796281e.jpg : A brown dog and a brown dog .
Predicting for image: 995
3727752439_907795603b.jpg : A young girl in a slide wearing her mouth .
Predicting for image: 996
3430607596_7e4f74e3ff.jpg : A child in a blue shirt is walking through a puddle of water .
Predicting for image: 997
3259666643_ae49524c81.jpg : A man in a red shirt is ready to make a sign .
Predicting for image: 998
2623930900_b9df917b82.jpg : A group of kids are all hugging .
Predicting for image: 999
3490736665_38710f4b91.jpg : A young boy jumps in the air with a football .
0.484078179369
